In [47]:
import numpy as np
import xgboost as xgb
import pandas as pd
# import category_encoders as ce
import sklearn
# from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
# import csv


In [72]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = train_data

train_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)

labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
train_data['sub_grade'] = labelencoder.fit_transform(train_data['sub_grade'])
train_data['home_ownership'] = labelencoder.fit_transform(train_data['home_ownership'])
train_data['emp_length'].replace('< 1 year', 0.5, inplace=True)
train_data['emp_length'].replace('1 year', 1.0, inplace=True)
train_data['emp_length'].replace('2 years', 2.0, inplace=True)
train_data['emp_length'].replace('3 years', 3.0, inplace=True)
train_data['emp_length'].replace('4 years', 4.0, inplace=True)
train_data['emp_length'].replace('5 years', 5.0, inplace=True)
train_data['emp_length'].replace('6 years', 6.0, inplace=True)
train_data['emp_length'].replace('7 years', 7.0, inplace=True)
train_data['emp_length'].replace('8 years', 8.0, inplace=True)
train_data['emp_length'].replace('9 years', 9.0, inplace=True)
train_data['emp_length'].replace('10 years', 10.0, inplace=True)
train_data['emp_length'].replace('10+ years', 15.0, inplace=True)
train_data['emp_length'] = train_data['emp_length'].fillna(0)

train_data['mort_acc'] = train_data['mort_acc'].fillna(0)
# Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]

X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_train.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0.0, inplace=True)
y_train_numeric.replace('Charged Off', 1.0, inplace=True)


## Adding artificial Data

In [93]:
print(type(X_train_numeric), type(y_train_numeric), type(X_test))

# print("BEFORE")
# print(X_train_numeric.shape)
# print(y_train_numeric.shape)
# print('AFTER')
# print(X_balanced.shape)
# print(Y_balanced.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [73]:
X_train_numeric.head()

,loan_amnt,term_(months),int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
0,5000,36,10.65,162.87,6,15.0,4,24000.0,27.65,3,0,13648,83.7,9,0.0,0.0
1,2500,60,15.27,59.83,13,0.5,4,30000.0,1.00,3,0,1687,9.4,4,0.0,0.0
2,2400,36,15.96,84.33,14,15.0,4,12252.0,8.72,2,0,2956,98.5,10,0.0,0.0
3,10000,36,13.49,339.31,10,15.0,4,49200.0,20.00,10,0,5598,21.0,37,0.0,0.0
4,3000,60,12.69,67.79,9,1.0,4,80000.0,17.94,15,0,27783,53.9,38,0.0,0.0


In [90]:
xg_train, xg_test, xg_ytrain, xg_ytest = sklearn.model_selection.train_test_split(
    X_train_numeric, y_train_numeric, test_size=0.2, random_state=0)

print(xg_train.shape, xg_test.shape, xg_ytrain.shape, xg_ytest.shape)
print(type(xg_train), type(xg_test), type(xg_ytrain), type(xg_ytest))

(157800, 16) (39450, 16) (157800,) (39450,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [112]:
X_balanced = []
Y_balanced = []
for i in range(xg_train.shape[0]):
    curr_df = xg_train.iloc[i,:]
    d = curr_df.to_dict()
    label = xg_ytrain[i]
    if label == 1:
        for _ in range(3):
            X_balanced.append(d)
            Y_balanced.append(label)
    X_balanced.append(d)
    Y_balanced.append(label)
xg_train = pd.DataFrame(X_balanced)
yg_train = pd.Series(Y_balanced)

KeyError: 5

In [61]:
def auc(m, xtrain, xtest): 
    return (metrics.roc_auc_score(xg_ytrain,m.predict_proba(xtrain)[:,1]),
                            metrics.roc_auc_score(xg_ytest,m.predict_proba(xtest)[:,1]))

import warnings
warnings.filterwarnings("ignore")

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10],
              "min_child_weight" : [2],
              "n_estimators": [200, 300],
              "learning_rate": [0.01, .1],
              "gamma" : [0,1]}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(xg_train, xg_ytrain)

print(grid_search.best_estimator_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[15:53:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=1, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=10, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=300, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [66]:
model = xgb.XGBClassifier(max_depth=10, min_child_weight=2,  n_estimators=250,
                          n_jobs=8, verbose=1,learning_rate=0.1, gamma=0, random_state=40, sampling_method = 'gradient_based')
model.fit(xg_train,xg_ytrain)
predictions1 = model.predict_proba(xg_test)[:,1]
# predictions2 = model.predict_proba(xg_test)
# print(predictions2)

print(auc(model, xg_train, xg_test))


[16:03:23] WARNING: ../src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:03:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
(0.9217113016083729, 0.6723766478761742)
